code in part inspired by: https://github.com/EthanRosenthal/torchmf

In [1]:
import os
import mxnet as mx
from mxnet import gluon, nd, ndarray

import pandas as pd
import numpy as np

In [55]:
data_path = '/tmp/ml-100k/'
prefix='ml-100k'
num_emb = 64
opt = 'Adam'
lr = 0.02
mmntm = 0.
wd = 0.
batch_size = 50
ctx = mx.gpu()
ctx = mx.cpu()

In [56]:
def download_ml_data(prefix, os_path):
    if not os.path.exists(os_path+"%s.zip" % prefix):
        print("Downloading MovieLens data: %s" % prefix)
        os.system("wget http://files.grouplens.org/datasets/movielens/{}.zip -P {}".format(prefix, data_path))
    print("unzip {}{}.zip -d {}".format(data_path,prefix, data_path))
    os.system("unzip {}{}.zip -d {}".format(data_path,prefix, data_path))   


download_ml_data(prefix=prefix, os_path=data_path)

unzip /tmp/ml-100k/ml-100k.zip -d /tmp/ml-100k/


In [57]:
def max_id(fname):
    mu = 0
    mi = 0
    with open(fname) as f:
        for line in f:
            tks = line.strip().split('\t')
            if len(tks) != 4:
                continue
            mu = max(mu, int(tks[0]))
            mi = max(mi, int(tks[1]))
    return mu + 1, mi + 1
max_users, max_items = max_id(data_path + prefix  +'/u.data')
(max_users, max_items)

(944, 1683)

In [58]:
train_df = pd.read_csv(data_path + prefix + '/u1.base', header=None, sep='\t')
test_df = pd.read_csv(data_path + prefix + '/u1.test', header=None, sep='\t')

train_data = nd.array(train_df[[0,1]].values, dtype=np.float32)
train_label = nd.array(train_df[2].values, dtype=np.float32)

test_data = nd.array(test_df[[0,1]].values, dtype=np.float32)
test_label = nd.array(test_df[2].values, dtype=np.float32)


In [59]:
class SparseMatrixDataset(gluon.data.Dataset):
    def __init__(self, data, label):
        assert data.shape[0] == len(label)
        self.data = data
        self.label = label
        if isinstance(label, ndarray.NDArray) and len(label.shape) == 1:
            self._label = label.asnumpy()
        else:
            self._label = label       
        
    def __getitem__(self, idx):
        return self.data[idx, 0], self.data[idx, 1], self.label[idx]
    
    def __len__(self):
        return self.data.shape[0]
        

In [60]:
class MFBlock(gluon.Block):
    def __init__(self, max_users, max_items, num_emb, dropout_p=0.5):
        super(MFBlock, self).__init__()
        
        self.max_users = max_users
        self.max_items = max_items
        self.dropout_p = dropout_p
        self.num_emb = num_emb
        
        with self.name_scope():
            self.user_embeddings = gluon.nn.Embedding(max_users, num_emb)
            self.item_embeddings = gluon.nn.Embedding(max_items, num_emb)
            self.dropout = gluon.nn.Dropout(dropout_p)
            self.dense = gluon.nn.Dense(num_emb, activation='relu')
            
    def forward(self, users, items):
        a = self.user_embeddings(users)
        a = self.dense(a)
        
        b = self.item_embeddings(items)
        b = self.dense(b)

        predictions = self.dropout(a) * self.dropout(b)      
        predictions = nd.sum(predictions, axis=1)
        return predictions

        

In [61]:
net = MFBlock(max_users=max_users, max_items=max_items, num_emb=num_emb, dropout_p=0.)
net.collect_params()

mfblock1_ (
  Parameter mfblock1_embedding0_weight (shape=(944, 64), dtype=float32)
  Parameter mfblock1_embedding1_weight (shape=(1683, 64), dtype=float32)
  Parameter mfblock1_dense0_weight (shape=(64, 0), dtype=float32)
  Parameter mfblock1_dense0_bias (shape=(64,), dtype=float32)
)

In [62]:
loss_function = gluon.loss.L2Loss()

In [63]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx, force_reinit=True)

In [64]:
trainer = gluon.Trainer(net.collect_params(), 'sgd',
                        {'learning_rate': lr, 'wd': wd, 'momentum': 0.9})

In [65]:
train_data_iter = gluon.data.DataLoader(SparseMatrixDataset(train_data, train_label), 
                                        shuffle=True, batch_size=batch_size)
test_data_iter = gluon.data.DataLoader(SparseMatrixDataset(test_data, test_label),
                                          shuffle=True, batch_size=batch_size)

In [66]:
def eval_net(data, net):
    acc = mx.metric.RMSE()
    for i, (user, item, label) in enumerate(data):
        user = user.as_in_context(ctx).reshape((batch_size,))
        item = item.as_in_context(ctx).reshape((batch_size,))
        label = label.as_in_context(ctx).reshape((batch_size,))
        predictions = net(user, item)
        loss = loss_function(predictions, label)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [67]:
eval_net(test_data_iter, net)


3.708269029855728

In [68]:
epochs = 10
#smoothing_constant = 10

def train(data_iter, net):
    for e in range(epochs):
        print("epoch: {}".format(e))
        for i, (user, item, label) in enumerate(train_data_iter):
            user = user.as_in_context(ctx).reshape((batch_size,))
            item = item.as_in_context(ctx).reshape((batch_size,))
            label = label.as_in_context(ctx).reshape((batch_size,))
            with mx.autograd.record():
                output = net(user, item)               
                loss = loss_function(output, label)
                loss.backward()
            net.collect_params().values()
            trainer.step(batch_size)
        print("EPOCH {}: RMSE ON TRAINING and TEST: {}. {}".format(e,
                                                                   eval_net(train_data_iter, net),
                                                                   eval_net(test_data_iter, net)))
    return "end of training"

In [ ]:
train(train_data_iter, net)

epoch: 0
EPOCH 0: RMSE ON TRAINING and TEST: 0.9558644323796034. 0.994868129491806
epoch: 1
EPOCH 1: RMSE ON TRAINING and TEST: 0.9496117281913757. 0.9851171465218067
epoch: 2


In [38]:
net1 = gluon.nn.Sequential()
with net1.name_scope():
    net1.add(gluon.nn.Embedding(max_users, num_emb))
    net1.add(gluon.nn.Dense(64))
    
net1.collect_params()

sequential6_ (
  Parameter sequential6_embedding0_weight (shape=(944, 64), dtype=<class 'numpy.float32'>)
  Parameter sequential6_dense0_weight (shape=(64, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential6_dense0_bias (shape=(64,), dtype=<class 'numpy.float32'>)
)

In [39]:
train_df = pd.read_csv(data_path+'u1.base', header=None, sep='\t')
test_df = pd.read_csv(data_path+'u1.test', header=None, sep='\t')

user_train_data = nd.array(train_df[[0]].values, dtype=np.float32)
item_train_data = nd.array(train_df[[1]].values, dtype=np.float32)
train_label = nd.array(train_df[2].values, dtype=np.float32)

user_test_data = nd.array(test_df[[0]].values, dtype=np.float32)
item_test_data = nd.array(test_df[[1]].values, dtype=np.float32)
test_label = nd.array(test_df[2].values, dtype=np.float32)



In [ ]:
user_train_data_iter = gluon.data.DataLoader(gluon.data.ArrayDataset())